# Notebook for Downloading Events of a Season

### Imports

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from sportradar_datacore_api import HandballAPI



### Configuration

In [2]:
NAME_COMPETITION = "1. Handball-Bundesliga"

NAME_SEASON = "DAIKIN HBL 2024/25"
PARAMETERS_SEASONS = {"startDate": "2024-01-01"}

### Initialize API

In [3]:
# Load credentials from .env_prd file
load_dotenv(".env", override=True)

# Initialize API client with environment-based credentials
api = HandballAPI(
    base_url=os.getenv("BASE_URL", ""),
    auth_url=os.getenv("AUTH_URL", ""),
    client_id=os.getenv("CLIENT_ID", ""),
    client_secret=os.getenv("CLIENT_SECRET", ""),
    org_id=os.getenv("CLIENT_ORGANIZATION_ID"),
    scopes=["read:organization"],
    sport="handball",
)

### Get wanted competition ID

In [4]:
competition = api.get_competitions(params={"nameLatinContains": NAME_COMPETITION})
display(competition)
competition_id = competition.get("data", [{}])[0].get("competitionId")

# Check if the competition was found
if not competition_id:
    raise ValueError(f"Competition '{NAME_COMPETITION}' not found.")
else:
    print(f"Competition ID: {competition_id}")

{'data': [{'abbreviationLatin': '',
   'abbreviationLocal': '',
   'added': '2024-07-03T16:07:08',
   'ageGroup': 'SENIOR',
   'competitionId': '4c445e5c-3956-11ef-9d0e-b74f5c057367',
   'eventType': 'FIXTURE',
   'externalId': '1',
   'images': [{'added': '2024-09-06T13:44:15',
     'baseId': '4c445e5c-3956-11ef-9d0e-b74f5c057367',
     'baseType': 'COMPETITION',
     'fileType': 'PNG',
     'imageId': '1b394bf9-6c56-11ef-9de2-2f11d3ac6058',
     'imageType': 'LOGO',
     'maximumHeight': 800,
     'maximumWidth': 1936,
     'organization': {'id': 'h1s44', 'resourceType': 'organizations'},
     'organizationId': 'h1s44',
     'secondaryType': None,
     'updated': '2024-09-06T13:44:15',
     'url': 'https://images.dc.prod.cloud.atriumsports.com/h1s44/8313948850f641df86e139645821ce6e'}],
   'internationalReference': '2',
   'nameLatin': '1. Handball-Bundesliga',
   'nameLocal': '1. Handball-Bundesliga',
   'organization': {'id': 'h1s44', 'resourceType': 'organizations'},
   'organizati

Competition ID: 4c445e5c-3956-11ef-9d0e-b74f5c057367


### Get wanted season ID

In [5]:
season = api.get_seasons(
    competition_id=competition_id, params=PARAMETERS_SEASONS
)
display(season)
season_id = season.get("data", [{}])[0].get("seasonId")
# Check if the season was found
if not season_id:
    raise ValueError(f"Season '{NAME_SEASON}' not found.")
else:
    print(f"Season ID: {season_id}")

{'data': [{'added': '2024-07-16T13:03:27',
   'ageGroup': 'SENIOR',
   'competition': {'id': '4c445e5c-3956-11ef-9d0e-b74f5c057367',
    'resourceType': 'competitions'},
   'competitionId': '4c445e5c-3956-11ef-9d0e-b74f5c057367',
   'discipline': None,
   'durationFull': 180,
   'endDate': '2026-06-30',
   'eventType': 'FIXTURE',
   'externalId': '245',
   'fixtureProfile': {'id': '0d1ad97e-4f5d-11ef-9f72-dfe4fac1221e',
    'resourceType': 'fixtureProfiles'},
   'gender': 'MALE',
   'grade': None,
   'images': [],
   'includeInStatistics': True,
   'internationalReference': None,
   'leaderCriteriaId': None,
   'liveDataAvailable': False,
   'liveVideoAvailable': False,
   'lockStandings': False,
   'nameLatin': None,
   'nameLocal': 'DAIKIN HBL 2024/25',
   'nameShortLatin': None,
   'nameShortLocal': None,
   'organization': {'id': 'h1s44', 'resourceType': 'organizations'},
   'organizationId': 'h1s44',
   'profileId': '0d1ad97e-4f5d-11ef-9f72-dfe4fac1221e',
   'promotionRelegationRu

Season ID: cabcf509-4373-11ef-a370-9d3c1e90234a


In [6]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, str):
            try:
                parsed = json.loads(json_data)
                self.json_str = json.dumps(parsed)
            except json.JSONDecodeError:
                raise ValueError("Invalid JSON string")
        else:
            self.json_str = json.dumps(json_data)
        self.uuid = str(uuid.uuid4())


    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### Get the fixtures (matches) of a season

In [7]:
import json
from IPython.display import JSON

season_fixtures = api.get_season_fixtures(season_id, params = {
        "include": "entities,organizations,persons,fixtures",
        "external": "entityId,personId",
        # "fields":"dob,firstName,organization(id),organizations[name], teams[name,details/metrics/*,tags(id)]",
        "limit": 1000,
    })
# print all keys in the response
print("Keys in the response:")
for key in season_fixtures.keys():
    print(key)
    value = season_fixtures[key]
    if isinstance(value, list):
        print(f"  Length list: {len(value)}")
    elif isinstance(value, dict):
        print(f"  Length dict: {len(value)}")

# Directly check for "includes" at top level
if "includes" in season_fixtures:
    print("Includes present at top level.")
    print("Includes keys:", list(season_fixtures["includes"].keys()))
    if "resources" in season_fixtures["includes"]:
        print("Resources found in includes:")
        for resource_key, resource_value in season_fixtures["includes"]["resources"].items():
            print(f"  {resource_key}: {len(resource_value)} items")
else:
    print("No 'includes' key found at top level.")


Keys in the response:
data
  Length list: 306
includes
  Length dict: 1
links
  Length dict: 1
meta
  Length dict: 9
Includes present at top level.
Includes keys: ['resources']
Resources found in includes:
  entities: 18 items
  fixtures: 306 items
  organizations: 1 items


In [15]:
# Convert fixture to dataframe
df_season_fixtures_data = pd.json_normalize(season_fixtures['data'], sep='_')
df_season_fixtures_includes_resources_entities = pd.DataFrame([
    {"key": key, **value} for key, value in season_fixtures.get('includes', {}).get('resources', {}).get('entities', {}).items()
])


print("season_fixtures['data']")
display(df_season_fixtures_data)
# # Convert the entities to a DataFrame
# df_season_fixtures_entities = pd.DataFrame([
#     {"key": key, **value} for key, value in season_fixtures.get('includes', {}).get('resources', {}).get('entities', {}).items()
# ])
print("season_fixtures['includes']['resources']['entities']")
display(df_season_fixtures_includes_resources_entities)



season_fixtures['data']


,added,attendance,broadcasts,competitorType,competitors,discipline,duration,durationFull,endTimeActualUTC,externalId,...,season_resourceType,stage_id,stage_resourceType,venue_id,venue_resourceType,fixtureProfile_id,fixtureProfile_resourceType,estimatedFinishTimeUTC,finishRecordingTimeLocal,finishRecordingTimeUTC
0,2024-07-16T13:04:57,9900.0,None,ENTITY,"[{'added': '2024-07-16T13:14:58', 'conferenceI...",None,None,NaN,None,57979,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d5550648-3952-11ef-9a4c-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN
1,2024-07-16T13:04:58,6600.0,None,ENTITY,"[{'added': '2024-07-16T13:14:59', 'conferenceI...",None,None,NaN,None,57980,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d51afda3-3952-11ef-a5b2-1bf414c1b522,venues,0d1ad97e-4f5d-11ef-9f72-dfe4fac1221e,fixtureProfiles,NaN,NaN,NaN
2,2024-07-16T13:04:59,NaN,None,ENTITY,"[{'added': '2024-07-16T13:15:00', 'conferenceI...",None,None,NaN,None,57981,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d5015bef-3952-11ef-8c12-1bf414c1b522,venues,0d1ad97e-4f5d-11ef-9f72-dfe4fac1221e,fixtureProfiles,NaN,NaN,NaN
3,2024-07-16T13:05:02,4246.0,None,ENTITY,"[{'added': '2024-07-16T13:15:03', 'conferenceI...",None,None,NaN,None,57982,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d4fa49b4-3952-11ef-b68b-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN
4,2024-07-16T13:05:02,3076.0,None,ENTITY,"[{'added': '2024-07-16T13:15:02', 'conferenceI...",None,None,NaN,None,57983,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d63b6098-3952-11ef-b196-1bf414c1b522,venues,0d1ad97e-4f5d-11ef-9f72-dfe4fac1221e,fixtureProfiles,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2024-07-16T13:04:52,NaN,None,ENTITY,"[{'added': '2024-07-16T13:14:54', 'conferenceI...",None,None,NaN,None,57974,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d4fdb7a5-3952-11ef-a3bf-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN
302,2024-07-16T13:04:52,NaN,None,ENTITY,"[{'added': '2024-07-16T13:14:54', 'conferenceI...",None,None,NaN,None,57975,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d4e8d781-3952-11ef-8041-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN
303,2024-07-16T13:04:54,NaN,None,ENTITY,"[{'added': '2024-07-16T13:14:56', 'conferenceI...",None,None,NaN,None,57977,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d51e9d4e-3952-11ef-841f-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN
304,2024-07-16T13:04:55,4200.0,None,ENTITY,"[{'added': '2024-07-16T13:14:55', 'conferenceI...",None,None,NaN,None,57976,...,seasons,cabcf509-4373-11ef-a370-9d3c1e90234a:1ee7e1,seasonStages,h1s44:d512f20b-3952-11ef-af51-1bf414c1b522,venues,NaN,NaN,NaN,NaN,NaN


season_fixtures['includes']['resources']['entities']


,key,added,additionalNames,address,ageGroup,alternateVenueIds,codeLatin,codeLocal,colors,contacts,...,internationalReference,nameFullLatin,nameFullLocal,organization,organizationId,representing,social,standard,status,updated
0,h1s44:0045fbc4-3953-11ef-a217-af5c55c3771d,2024-07-03T15:43:32,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Hellgrundweg 50', 'city': 'Hambu...",SENIOR,"[d63b6098-3952-11ef-b196-1bf414c1b522, d5d7128...",HSV,HSV,"{'primary': 'ff0000', 'secondary': None, 'tert...","{'email': 'info@hamburg-handball.de', 'fax': '...",...,None,Handball Sport Verein Hamburg,Handball Sport Verein Hamburg,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/hamburg...,None,ACTIVE,2025-05-25T04:26:06
1,h1s44:fe73c376-3952-11ef-8a18-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Wolfgang-Kühle-Straße 1', 'city'...",SENIOR,[],WET,WET,"{'primary': '046e09', 'secondary': None, 'tert...","{'email': 'info@hsg-wetzlar.de', 'fax': '+49 6...",...,None,HSG Wetzlar,HSG Wetzlar,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/HSGWetz...,None,ACTIVE,2025-05-25T04:21:38
2,h1s44:fe7bdd16-3952-11ef-b585-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Markgrafenstr. 34', 'city': 'Ber...",SENIOR,None,BER,BER,"{'primary': '4f7d4b', 'secondary': None, 'tert...","{'email': 'info@fuechse.berlin', 'fax': '+4930...",...,None,Füchse Berlin,Füchse Berlin,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/fuechse...,None,ACTIVE,2025-05-25T04:21:54
3,h1s44:fe80598a-3952-11ef-914c-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Franz-Grashof-Straße 5-7', 'city...",SENIOR,None,RNL,RNL,"{'primary': None, 'secondary': None, 'tertiary...","{'email': 'info@rhein-neckar-loewen.de', 'fax'...",...,None,Rhein-Neckar Löwen,Rhein-Neckar Löwen,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/rnloewe...,None,ACTIVE,2025-05-25T04:22:08
4,h1s44:fe848316-3952-11ef-8185-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Heinz-Krügel-Platz 3', 'city': '...",SENIOR,None,SCM,SCM,"{'primary': '000000', 'secondary': None, 'tert...","{'email': 'handball@sc-magdeburg.de', 'fax': '...",...,None,SC Magdeburg,SC Magdeburg,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/SGFleHa...,None,ACTIVE,2025-05-25T04:22:23
5,h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Schiffbrücke 66', 'city': 'Flens...",SENIOR,None,SGF,SGF,"{'primary': 'ff0000', 'secondary': None, 'tert...","{'email': 'info@sg-flensburg-handewitt.de', 'f...",...,None,SG Flensburg-Handewitt,SG Flensburg-Handewitt,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/SGFleHa...,None,ACTIVE,2025-05-25T04:22:41
6,h1s44:fe8d1885-3952-11ef-9130-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Nägelsbachstraße 33', 'city': 'E...",SENIOR,[],HCE,HCE,"{'primary': '0c0c0d', 'secondary': None, 'tert...","{'email': 'info@hc-erlangen.de', 'fax': '+4991...",...,None,HC Erlangen,HC Erlangen,"{'id': 'h1s44', 'resourceType': 'organizations'}",h1s44,None,{'facebook': 'https://www.facebook.com/hc.erla...,None,ACTIVE,2025-05-25T04:22:58
7,h1s44:fe911367-3952-11ef-9131-af5c55c3771d,2024-07-03T15:43:29,"{'namePlaceLatin': None, 'namePlaceLocal': Non...","{'address1': 'Steinmüllerallee 3', 'city': 'Gu...",SENIOR,None,GUM,GUM,"{'primary': '086bff', 'secondary': None, 'tert...","{'email': 'info@vfl-gummersbach.de', 'fax': '+...",...,None,VfL Gummersbach,VfL Gummersbach,"{'id': 'h1s44', 'resourceType

### Get Play-by-Play (Event Timeline) for every match in the season

In [ ]:
play_by_play_exports = []

for index, row in df_season_fixtures_data.iterrows():
    fixture_id = row['fixtureId']

    # get more info about fixture
    fixture = api.get_fixture(fixture_id, params={
        "include": "entities,organizations,persons",
        "external": "entityId,personId",
    })
    df_fixture = pd.json_normalize(fixture["data"], sep='_')
    df_fixture_include = pd.json_normalize(fixture.get("includes", {}).get("resources", {}), sep='_')
    display(df_fixture)

    fixture_playbyplay_export = api.get_match_events_export(fixture_id, params={
            "include": "entities,organizations,persons,fixtures",
            "external": "entityId,personId",
            "withScores": "true",
        },)
    
    fixture_timeline = fixture_playbyplay_export.get("data", [{}])
    df_fixture_timeline = pd.json_normalize(fixture_timeline, sep='_')
    display(df_fixture_timeline)

    list_entities_match = df_fixture_timeline["data_entityId"].unique().tolist()
    # remove nans
    list_entities_match = [e for e in list_entities_match if pd.notna(e)]

    # filter entities from the season fixtures in column entityId
    df_match_entities = df_season_fixtures_includes_resources_entities[
        df_season_fixtures_includes_resources_entities["entityId"].isin(list_entities_match)
    ]
    # unravel json items in the columns
    df_match_entities = pd.json_normalize(df_match_entities.to_dict(orient='records'), sep='_')
    display(df_match_entities)

    fixture_gameday = row.get("roundNumber", "")
    
    events = []
    for event in fixture_timeline:
        scores = event.get("scores", {})
        flat = {
            **{k: v for k, v in event.items() if k not in ["data", "scores"]},
            **{f"data.{k}": v for k, v in event.get("data", {}).items()},
            **{f"data.options.{k}": v for k, v in event.get("data", {}).get("options", {}).items()},
        }
        flat["raw_scores"] = scores
        events.append(flat)

    df = pd.DataFrame(events)

    df_entities = df_season_entities
    df_entities["entityId"] = df_entities["entityId"]
    df_entities["abbreviation"] = df_entities["codeLocal"]

    persons = fixture.get("resources", {}).get("persons", {})
    df_persons = pd.DataFrame.from_dict(persons, orient="index")
    df_persons["personId"] = df_persons["personId"]

    df = df.merge(
        df_entities[["entityId", "nameFullLocal", "abbreviation"]],
        left_on="data.entityId",
        right_on="entityId",
        how="left"
    )
    df = df.merge(
        df_persons[["personId", "nameFullLocal"]],
        left_on="data.personId",
        right_on="personId",
        how="left",
        suffixes=("_entity", "_person"),
    )

    df = df.rename(columns={
        "nameFullLocal_entity": "team_name",
        "nameFullLocal_person": "player_name",
        "abbreviation": "team_abbreviation"
    })

    competitors = fixture.get("competitors", [])
    home_team = next(c for c in competitors if c.get("isHome"))
    away_team = next(c for c in competitors if not c.get("isHome"))

    home_team_id = home_team["entityId"]
    away_team_id = away_team["entityId"]

    home_team_name = df_entities.loc[df_entities["entityId"] == home_team_id, "nameFullLocal"].iloc[0]
    away_team_name = df_entities.loc[df_entities["entityId"] == away_team_id, "nameFullLocal"].iloc[0]
    home_team_abbr = df_entities.loc[df_entities["entityId"] == home_team_id, "abbreviation"].iloc[0]
    away_team_abbr = df_entities.loc[df_entities["entityId"] == away_team_id, "abbreviation"].iloc[0]

    df["team_home_abbr"] = home_team_abbr
    df["team_away_abbr"] = away_team_abbr
    df["team_home_id"] = home_team_id
    df["team_home_name"] = home_team_name
    df["team_away_id"] = away_team_id
    df["team_away_name"] = away_team_name
    df["team_attacking_id"] = df["data.entityId"]
    df["team_attacking_name"] = df["team_name"]
    df["gameday"] = fixture_gameday

    df["score_home"] = df.apply(
        lambda row: row["raw_scores"].get(home_team_id) if isinstance(row["raw_scores"], dict) else None,
        axis=1
    )
    df["score_away"] = df.apply(
        lambda row: row["raw_scores"].get(away_team_id) if isinstance(row["raw_scores"], dict) else None,
        axis=1
    )
    df["attacking_side"] = df["team_attacking_id"].apply(
        lambda x: "home" if x == home_team_id else "away" if x == away_team_id else "unknown"
    )

,data,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_added,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_additionalNames_namePlaceLatin,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_additionalNames_namePlaceLocal,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_additionalNames_nameShortLatin,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_additionalNames_nameShortLocal,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_address_address1,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_address_city,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_address_countryCode,includes_resources_entities_h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d_address_postalCode,...,links_self,meta_code,meta_codeVersion,meta_count,meta_fromCache,meta_generationTime,meta_limit,meta_offset,meta_time,meta_version
0,"[{'added': '2024-07-16T13:04:57', 'attendance'...",2024-07-03T15:43:29,None,None,None,None,Schiffbrücke 66,Flensburg,DEU,24939.0,...,https://api.dc.connect.sportradar.com/v1/handb...,200,d6cd1e2bd19e03a81132a23b2025920577f84e37,1,False,0.172368,10,0,2025-05-25T09:50:59.425731Z,1


,clientId,clientType,fixtureId,organizationId,received,sport,topic,type,data_class,data_eventId,...,data_options_attackType,data_options_goalKeeperId,data_options_location,data_success,data_x,data_y,data_options_failureReason,data_flagged,data_options_value,data_options_emptyNet
0,0b623a10-be2f-11ef-93f8-2c6dc192e163,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734629741301,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,setup,b7d8cd90-8ed2-11ef-b16d-cda25f1166cf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0b623a10-be2f-11ef-93f8-2c6dc192e163,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734629741328,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,setup,62917f30-8ed7-11ef-b16d-cda25f1166cf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0b623a10-be2f-11ef-93f8-2c6dc192e163,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734629741343,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,setup,6591c6e0-8ed7-11ef-b16d-cda25f1166cf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0b623a10-be2f-11ef-93f8-2c6dc192e163,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734629741370,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,setup,6e1557a0-8ed7-11ef-b16d-cda25f1166cf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0b623a10-be2f-11ef-93f8-2c6dc192e163,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734629741410,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,setup,708c9930-8ed7-11ef-b16d-cda25f1166cf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,s-mWzZ9cTnZYHMarpTVZeSXN,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734632308729,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,sport,a542b051-be35-11ef-9482-850ba4c8b178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,s-mWzZ9cTnZYHMarpTVZeSXN,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734632308787,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,sport,a54b14c0-be35-11ef-9482-850ba4c8b178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,s-mWzZ9cTnZYHMarpTVZeSXN,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734632309760,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,sport,a5ddb7d0-be35-11ef-9482-850ba4c8b178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,s-mWzZ9cTnZYHMarpTVZeSXN,InGame:0.9,00c08679-4374-11ef-80bd-73cf0bc66b45,h1s44,1734632313482,h,s/h/h1s44/00c08679-4374-11ef-80bd-73cf0bc66b45...,event,sport,a8177b30-be35-11ef-9482-850ba4c8b178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,key,added,ageGroup,alternateVenueIds,codeLatin,codeLocal,defaultVenueId,discipline,entityGroupId,entityId,...,entityGroup_id,entityGroup_resourceType,organization_id,organization_resourceType,social_facebook,social_instagram,social_twitter,social_website,social_ticketing,social_tiktok
0,h1s44:fe88f93b-3952-11ef-aa5d-af5c55c3771d,2024-07-03T15:43:29,SENIOR,None,SGF,SGF,d529a583-3952-11ef-bb2c-1bf414c1b522,None,ded85825-3952-11ef-baea-559511c85b53,fe88f93b-3952-11ef-aa5d-af5c55c3771d,...,h1s44:ded85825-3952-11ef-baea-559511c85b53,entityGroups,h1s44,organizations,https://www.facebook.com/SGFleHa/,https://www.instagram.com/sgflensburghandewitt/,https://twitter.com/SGFleHa,http://www.sg-flensburg-handewitt.de/,NaN,NaN
1,h1s44:fea93a14-3952-11ef-a7e0-af5c55c3771d,2024-07-03T15:43:29,SENIOR,[d53b5293-3952-11ef-9c22-1bf414c1b522],HAN,HAN,d5550648-3952-11ef-9a4c-1bf414c1b522,None,ebdb5538-3952-11ef-a5a5-85d73a57ae73,fea93a14-3952-11ef-a7e0-af5c55c3771d,...,h1s44:ebdb5538-3952-11ef-a5a5-85d73a57ae73,entityGroups,h1s44,organizations,https://www.facebook.com/DIERECKEN/,https://www.instagram.com/dierecken/,,http://www.die-recken.de/home/,https://www.eventimsports.de/ols/recken/,https://www.tiktok.com/@dierecken


NameError: name 'df_season_entities' is not defined

In [100]:
# for fixture in season_fixtures["data"]:
#     id_fixture = fixture.get("fixtureId")
#     # Check if the fixture was found
#     if not id_fixture:
#         raise ValueError(f"Fixture not found.")
#     else:
#         print(f"Fixture ID: {id_fixture}")

#     querystring = {
#         "external": "entityId,personId",
#         # "fields": "dob,firstName,organization(id),organizations[name], teams[name,details/metrics/*,tags(id)]",
#         "hideNull": "true",
#         "include": "organizations,fixtures,entities",
#         "withScores": "true",
#     }

#     # Get fixture details
#     fixture_playbyplay_export = api.get_match_events_export(
#         id_fixture,
#         params={
#             "include": "entities,organizations,fixtures,persons",
#             "external": "entityId,personId",
#             "withScores": "true",
#         },
#         # params=querystring,
#     )

#     fixture_playbyplay = api.get_playbyplay(id_fixture, params={
#         "include": "entities,organizations,persons",
#         "external": "entityId,personId",
#     })

#     # Check if the fixture details were found
#     if not fixture_playbyplay_export:
#         raise ValueError(f"Fixture details not found.")

#     # Extract key fields into a flattened structure for easier inspection
#     fixture_data = fixture_playbyplay_export.get("data", [{}])
#     fixture_includes = fixture_playbyplay.get("includes", {})

#     fixture_gameday = fixture.get("roundNumber", {})

#     events = []
#     for event in fixture_data:
#         scores = event.get("scores", {})

#         flat = {
#             **{k: v for k, v in event.items() if k not in ["data", "scores"]},
#             **{f"data.{k}": v for k, v in event.get("data", {}).items()},
#             **{
#                 f"data.options.{k}": v
#                 for k, v in event.get("data", {}).get("options", {}).items()
#             },
#         }

#         flat["raw_scores"] = scores
#         events.append(flat)

#     df = pd.DataFrame(events)

#     entities = fixture_includes.get("resources", {}).get("entities", {})
#     df_entities = pd.DataFrame.from_dict(entities, orient="index")
#     df_entities["entityId"] = df_entities["entityId"]
#     df_entities["abbreviation"] = df_entities["codeLocal"]

#     persons = fixture_includes.get("resources", {}).get("persons", {})
#     df_persons = pd.DataFrame.from_dict(persons, orient="index")
#     df_persons["personId"] = df_persons["personId"]

#     df = df.merge(
#         df_entities[["entityId", "nameFullLocal", "abbreviation"]],
#         left_on="data.entityId",
#         right_on="entityId",
#         how="left"
#     )
#     df = df.merge(
#         df_persons[["personId", "nameFullLocal"]],
#         left_on="data.personId",
#         right_on="personId",
#         how="left",
#         suffixes=("_entity", "_person"),
#     )

#     df = df.rename(
#         columns={
#             "nameFullLocal_entity": "team_name",
#             "nameFullLocal_person": "player_name",
#             "abbreviation": "team_abbreviation"
#         }
#     )

#     # Extract home/away teams from fixture metadata
#     competitors = fixture.get("competitors", [])
#     home_team = next(c for c in competitors if c.get("isHome"))
#     away_team = next(c for c in competitors if not c.get("isHome"))

#     home_team_id = home_team["entityId"]
#     away_team_id = away_team["entityId"]

#     home_team_name = df_entities.loc[df_entities["entityId"] == home_team_id, "nameFullLocal"].iloc[0]
#     away_team_name = df_entities.loc[df_entities["entityId"] == away_team_id, "nameFullLocal"].iloc[0]

#     home_team_abbr = df_entities.loc[df_entities["entityId"] == home_team_id, "abbreviation"].iloc[0]
#     away_team_abbr = df_entities.loc[df_entities["entityId"] == away_team_id, "abbreviation"].iloc[0]

#     df["team_home_abbr"] = home_team_abbr
#     df["team_away_abbr"] = away_team_abbr



#     df['team_home_id'] = home_team_id
#     df['team_home_name'] = home_team_name
#     df['team_away_id'] = away_team_id
#     df['team_away_name'] = away_team_name
#     df['team_attacking_id'] = df['data.entityId']
#     df['team_attacking_name'] = df['team_name']
#     df['gameday'] = fixture_gameday

#     df['score_home'] = df.apply(
#         lambda row: row['raw_scores'].get(row['team_home_id']) if isinstance(row['raw_scores'], dict) else None,
#         axis=1
#     )
#     df['score_away'] = df.apply(
#         lambda row: row['raw_scores'].get(row['team_away_id']) if isinstance(row['raw_scores'], dict) else None,
#         axis=1
#     )

#     df['attacking_side'] = df['team_attacking_id'].apply(
#         lambda x: 'home' if x == home_team_id else 'away' if x == away_team_id else 'unknown'
#     )

#     # save to csv with naming scheme YYYY-MM-DD_gameday_GAMEDAY_id_FIXTURE-ID_ABBREVIATION-HOME-ABBREVIATION-AWAY.csv
#     fixture_date = fixture.get("startTimeLocal", {}).split("T")[0]
#     fixture_gameday = f"{int(fixture.get('roundNumber', 0)):02}"
#     fixture_id = fixture.get("fixtureId", {})

#     fixture_abbr = f"{home_team_abbr}-{away_team_abbr}"
#     fixture_abbr = fixture_abbr.replace("-", "_")

#     name_file = f"{fixture_date}_gameday_{fixture_gameday}_id_{fixture_id}_{fixture_abbr}.csv"
#     path = os.path.join("../data", name_file)
#     df.to_csv(path, index=False)
#     print(f"Fixture data saved to {path}")






#     # break

In [ ]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


for fixture in season_fixtures["data"].items():
#     process_fixture(fixture)
# def process_fixture(fixture):
    id_fixture = fixture.get("fixtureId")
    if not id_fixture:
        continue  # Skip if fixture ID is not found

    try:
        fixture_gameday = fixture.get("roundNumber", {})
        fixture_date = fixture.get("startTimeLocal", {}).split("T")[0]
        fixture_id = fixture.get("fixtureId", {})

        # Get fixture details
        fixture_playbyplay_export = api.get_match_events_export(
            id_fixture,
            params={
                "include": "entities,organizations,fixtures,persons",
                "external": "entityId,personId",
                "withScores": "true",
            },
        )

        fixture_playbyplay = api.get_playbyplay(id_fixture, params={
            "include": "entities,organizations,persons",
            "external": "entityId,personId",
        })

        if not fixture_playbyplay_export:
            # return
            continue

        fixture_timeline = fixture_playbyplay_export.get("data", [{}])
        fixture_includes = fixture_playbyplay.get("includes", {})

        events = []
        for event in fixture_timeline:
            scores = event.get("scores", {})
            flat = {
                **{k: v for k, v in event.items() if k not in ["data", "scores"]},
                **{f"data.{k}": v for k, v in event.get("data", {}).items()},
                **{f"data.options.{k}": v for k, v in event.get("data", {}).get("options", {}).items()},
            }
            flat["raw_scores"] = scores
            events.append(flat)

        df = pd.DataFrame(events)

        entities = fixture.get("resources", {}).get("entities", {})
        df_entities = pd.DataFrame.from_dict(entities, orient="index")
        df_entities["entityId"] = df_entities["entityId"]
        df_entities["abbreviation"] = df_entities["codeLocal"]

        persons = fixture.get("resources", {}).get("persons", {})
        df_persons = pd.DataFrame.from_dict(persons, orient="index")
        df_persons["personId"] = df_persons["personId"]

        df = df.merge(
            df_entities[["entityId", "nameFullLocal", "abbreviation"]],
            left_on="data.entityId",
            right_on="entityId",
            how="left"
        )
        df = df.merge(
            df_persons[["personId", "nameFullLocal"]],
            left_on="data.personId",
            right_on="personId",
            how="left",
            suffixes=("_entity", "_person"),
        )

        df = df.rename(columns={
            "nameFullLocal_entity": "team_name",
            "nameFullLocal_person": "player_name",
            "abbreviation": "team_abbreviation"
        })

        competitors = fixture.get("competitors", [])
        home_team = next(c for c in competitors if c.get("isHome"))
        away_team = next(c for c in competitors if not c.get("isHome"))

        home_team_id = home_team["entityId"]
        away_team_id = away_team["entityId"]

        home_team_name = df_entities.loc[df_entities["entityId"] == home_team_id, "nameFullLocal"].iloc[0]
        away_team_name = df_entities.loc[df_entities["entityId"] == away_team_id, "nameFullLocal"].iloc[0]
        home_team_abbr = df_entities.loc[df_entities["entityId"] == home_team_id, "abbreviation"].iloc[0]
        away_team_abbr = df_entities.loc[df_entities["entityId"] == away_team_id, "abbreviation"].iloc[0]

        df["team_home_abbr"] = home_team_abbr
        df["team_away_abbr"] = away_team_abbr
        df["team_home_id"] = home_team_id
        df["team_home_name"] = home_team_name
        df["team_away_id"] = away_team_id
        df["team_away_name"] = away_team_name
        df["team_attacking_id"] = df["data.entityId"]
        df["team_attacking_name"] = df["team_name"]
        df["gameday"] = fixture_gameday

        df["score_home"] = df.apply(
            lambda row: row["raw_scores"].get(home_team_id) if isinstance(row["raw_scores"], dict) else None,
            axis=1
        )
        df["score_away"] = df.apply(
            lambda row: row["raw_scores"].get(away_team_id) if isinstance(row["raw_scores"], dict) else None,
            axis=1
        )
        df["attacking_side"] = df["team_attacking_id"].apply(
            lambda x: "home" if x == home_team_id else "away" if x == away_team_id else "unknown"
        )

        fixture_abbr = f"{home_team_abbr}-{away_team_abbr}".replace("-", "_")
        fixture_gameday_str = f"{int(fixture_gameday):02}"
        name_file = f"{fixture_date}_gameday_{fixture_gameday_str}_id_{fixture_id}_{fixture_abbr}.csv"
        path = os.path.join("../data", name_file)
        df.to_csv(path, index=False)
        print(f"Saved: {path}")
    except Exception as e:
        print(f"Failed to process fixture {id_fixture}: {e}")


# Run in a normal loop
# for fixture in season_fixtures["data"]:
#     process_fixture(fixture)

# Run in parallel
# with ThreadPoolExecutor(max_workers=12) as executor:
#     futures = [executor.submit(process_fixture, f) for f in season_fixtures["data"]]
#     for future in as_completed(futures):
#         future.result()